### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


In [3]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [4]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [5]:
head(df,2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,100,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
df$flag <- FALSE
df$true_false <- FALSE


In [7]:
head(df, 2)

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.0700,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


## <font color = yellow>1. Create data & dfs

In [8]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

create_data()
sale_records <- data.frame()

In [9]:
dim(df)


[1] 41 23

In [10]:
sale_records <- data.frame()

## <font color = lime> Examining data and dfs

In [11]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 3)
head(sells,3)

[1] 23

[1] 18

[1] 18 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
5,5,2023-01-23,Buy,AMC,1000,5.788,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.51240,5512.40,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
6,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [12]:
dim(b)
b

[1]  1 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


### <font color = yellow>2. update_b() - updates the basic b variable with data from buy/sell rows 
    
#### b>s, b<s, b==s,

In [13]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


In [14]:
last_row <- tail(buys, n = 1)
last_row

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
39,39,2023-05-17,Buy,AMC,1000,5.009,5009,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [15]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function(b,s, buys) {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE 
            buys$flag[1] <- TRUE
            
            
            # All s will be eliminated (Sold) in this transaction
            s$type <- "SOLD"
            cat("POC - updated_s when b > s: ", s$type, "\n")
    
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {   
            cat("proof 3 \n")
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # need a new flag setting for update_buy_sells fct
            if(b$type == "Buy CF" && s$type == "Sell CF") {
        cat("proof \n")
                b$co <- "CFCF"
                b$true_false <- TRUE
                s$tru_false <- TRUE
                } 
        
            s$type <- "EQ SOLD"
            cat("POC - updated_s when b == s: ", s$type, "\n")            
            }
            
        # s$type <- "SOLD"
        # cat("POC - updated_s : ", s$type, "\n")
        return(list(b,s, buys)) 
     
  }  # fct

# updated_b_list <- update_b(b,s,buys)

# updated_b <- updated_b_list[[1]]
# updated_s <- updated_b_list[[2]]
# buys <- updated_b_list[[3]]

# dim(updated_b)
# dim(buys)
    

In [16]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

dim(updated_b)
dim(buys)
    

[1]  1 23

[1] 23 23

In [17]:
updated_s
updated_b
b[1,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [18]:
updated_b

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


### <font color = yellow>3. create new sales_records (rbind) with the updated_b 

In [19]:
# df_name <- "iris"  # Replace with your data frame name
# tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

In [20]:
# df_name <- "sale_records"
# tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

In [21]:
# if (exists("sale_records") && is.data.frame(get("sale_records"))) {
#   # Data frame exists and is a valid data frame
#   # Your code here...
#     cat("sale_records exists:     ", "\n") 
# } else {
#   # Data frame does not exist or is not a valid data frame
#   # Handle the case where the data frame is missing...
#     cat("Newly created sale_records with updated_b data. ", "\n")
#     sale_records <- data.frame()
#     sale_records <- rbind(sale_records, updated_b)    
#     # sale_records <- add_row(updated_b ) # , .before = 1)
# }
# sale_records
        
        

In [22]:
# First go around needs rbind()
sale_records <- rbind(sale_records, updated_b)    

### <font color = yellow> 4. update_buys_sells() 
### <font color = yellow>  update buys or sells (add_row) when c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


#### <font color = lime> How it's done

In [23]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


In [24]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [25]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_buys_sells <- function(buys, sells) {

    #  Check the most recent row ??? of sale_record or the last row of sale_record (was bind() or add_row() ? )
    
   last_row <- head(sale_records, n = 1)
   
    #first_row <- head(sale_records, n=1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    # both start with the same inputs from the sale_records
    b1 <- last_row
    s1 <- last_row
    
    
    
    #  b > s  ... buy c/f will be created 
   
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        
        
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
       
        
        
   
        
        
# -------------------------------------------------------------------------------------------------------------------------   
        
        
        # NEW  ... This section makes changes in sells[2, ]  which becomes sells[3,] , due to add_row()
                
        
        s2cf_quan <- sells$quan[2] - s1$sold_cf_quan
        s2cf_pps <- sells$pps[2]
        
        cat("the s2cf_quan is: ", s2cf_quan, "\n")
        cat("the s2cf_pps is ", s2cf_pps, "\n")
        
        sells$quan[2] <- s2cf_quan
        sells$pps[2] <-  s1$sell_pps                     #    s2cf_pps 
        sells[2, ]
        sells$basis[2] <- sells$pps[2] * s2cf_quan
        
        
       
        
# -------------------------------------------------------------------------------------------------------------------------    
        
        
        #  The section updates b1   .... t/f buys
        
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- TRUE
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        
      #  test <- test %>% add_row(b1, .before = 1)
        test <- test %>% add_row(b1, .after = 1)
        buys <- test
        
  # ---------------------------------------------------------------------------------------------------------------      
        
               
        # this section updates s1 ... t/f sells
       
        
            
        last_row$co <- "c/f"    
        
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <-  s1$sell_pps               #    s1$sold_cf_pps
        s1$basis <- s1$sell_pps
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
 
        
        
# =================================================================================================================
# ==============================================================================================================
 # =================================================================================================================
# ==============================================================================================================
              
        
             #   s > b ... sell carryforward will be created    
        

    
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
        
        
       
        cat("\n", "There is Sold carryback. Update sells carryback -2 ", last_row$sold_cf_quan, "\n")
           # cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/b"      
       
        s1$type <- "Sell CB"
        s1$quan <- s1$sold_cf_quan
        s1$pps <- sells$pps[1]
        s1$basis <- s1$sold_cf_basis
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .after = 1)
        sells <- test
 
    # =================================================================================================================
# ==============================================================================================================
       
  # =================================================================================================================
# ==============================================================================================================
       
        ## must also update the buys to match that sells c/f (this will allow for loop2 to begin 
        ## with both the buy and sell processing the c/f (example when sell cf = 900, then the 
        ## buy row needs to be that same 900 .... do not delete, just update here and now 
        
        b1$type <- "Buy CB"
        # b1$quan <- b1$sold_cf_quan
       # b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- FALSE
        
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0
        
        test2 <- buys
    #    test2 <- test2 %>% add_row(b1, .before = 1)
        buys <- test2
                      
        }

  # =================================================================================================================
# ==============================================================================================================
# =================================================================================================================
# ==============================================================================================================
         
    
    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ], dim of buys increases with c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... dim of sells increases with c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_buys_sells. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
# return_list <- update_buys_sells(buys, sells)

# buys <- return_list[[1]]
# sells <- return_list[[2]]

# dim(buys)
# dim(sells)

In [26]:
# fct call
return_list <- update_buys_sells(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


 There is Sold carryback. Update sells carryback -2  900 
printing: s1 ... dim of sells increases with c/f  


[1] 23 23

[1] 19 23

#### <font color = yellow>5. Snip(), delete row from buys and sells / replace row when c/f 

In [27]:
range(x <- sort(round(stats::rnorm(10) - 1.2, 1)))

if(any(x < 0)) cat("x contains negative values\n")

[1] -2.4  0.2

x contains negative values


In [28]:
if(any(buys[1,] == TRUE)) cat("buys contains cf \n")

buys contains cf 


In [29]:
buys[1:3,]
sells[1:3, ]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.070,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
5,5,2023-01-23,Buy,AMC,1000,5.788,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2,2023-01-27,Sell,AMC,1000,5.51240,5512.40,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1,2022-12-31,Sell CB,AMC,900,5.51240,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


In [30]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [31]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (b, s, buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    
       
    # snip .... if 
        
    if(buys$quan[1] == sells$quan[1] && buys$type[1] == "Buy CF" && sells$type[1] == "Sell CF")  { #  sells$type[1]) {
        cat("new condtional works for filtering in copy_snip when cf-cf \n")
        buys <-  buys[-1, ]
        sells <- sells[-1, ]    
     return(list(b1, s1, buys, sells))  
        
  #  if(buys$type[1] == "Buy CF")   { 
    } else if (buys$type[1] == "Buy CF" || buys$flag[1] == TRUE ) {   #  && buys$true_false == FALSE)   {         
   # if(buys$flag[1] == "TRUE")   {         
        cat(" Contains buy carryforward - remove buys c/f row and DO snip sells \n")         
         # remove the original buys and sells
         buys <- buys[-1, ]    
         sells <- sells[-2, ]
        return(list(b1, s1, buys, sells))                   
        
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))
# new       
    } else if (sells$type[2] == "Sell CB") {
        cat("sells contains carryback .. Remove sells[1,]", "\n") 
        sells <- sells[-1,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))
        
     }else if (buys$co[1] == "CFCF" ) {
        cat("this is a cf/cf wash sales ... replace buys[1,] sells[1,] \n")
        buys <- buys[-1]
        sells <- sells[-1]
         return(list(b1, s1, buys, sells))
        
    } else if(b1$flag == FALSE & b1$true_false == FALSE) {
        
        cat("check false / false \n")        
        buys <- buys[-1,]
        sells <- sells[-1,]
        
        return(list(b1, s1, buys, sells))
        
    } else {
        cat("There is no carryforward in buys or sells, This msgs means there is an issue. \n")
        # buys <- buys[-1,]
        # sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
 
 } # fct     
   
# fct call passing 2 arguments
# copy_snip_list <- copy_snip(b, s, buys, sells) 

# b1 <- copy_snip_list[[1]]
# s1 <- copy_snip_list[[2]]
# buys <- copy_snip_list[[3]]
# sells <- copy_snip_list[[4]]

# # after
# dim(buys)
# dim(sells)
# # b1
# s1

In [32]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

sells contains carryback .. Remove sells[1,] 


[1] 22 23

[1] 18 23

In [33]:
b1

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,1,2022-12-31,Buy,AMC,100,4.07,4273.5,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


#### <font color = lime> QC: Loop 1 review buys & sells

In [34]:
buys[1:4,]
sells[1:4,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.6250,5625.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
5,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
7,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
9,9,2023-02-10,Buy,AMC,2000,4.7450,9490.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,1,2022-12-31,Sell CB,AMC,900,5.51240,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,8,2023-02-03,Sell,AMC,1000,6.19010,6190.10,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop 1 review sale_record for accuracy

In [35]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [36]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} 

sale_records exists:      


In [37]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,2022-12-31,Buy,AMC,100,4.07,407,100,FALSE,0,⋯,4.07,407,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


In [38]:
# stopps #  Old code

# <font color = cyan> Proof of Concept above. End of loop 1 

#### <font color = yellow>1. Create data & dfs

In [39]:
b <- buys[1,]
s <- sells[1,]

In [40]:
b
s

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,1,2022-12-31,Sell CB,AMC,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [41]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

dim(updated_b)
dim

POC - updated_s when b > s:  SOLD 


[1]  1 23

function (x)  .Primitive("dim")

### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [42]:
b

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.625,5625,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


In [43]:
sale_records <- sale_records %>% add_row(updated_b , .before = 1)
sale_records[1,]
sale_records

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
1,3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.625,562.5,FALSE


index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.6250,562.50,FALSE
1,2022-12-31,Buy,AMC,100,4.070,407,100,FALSE,0,⋯,4.070,407.0,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


## <font color = lime> CAUTION: Step3 using either before/after, will effect Step 4 below:  

In [44]:
buys[1:4,]
sells[1:4,]

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,3,2023-01-20,Buy,AMC,1000,5.6250,5625.0,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
5,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
7,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE
9,9,2023-02-10,Buy,AMC,2000,4.7450,9490.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,1,2022-12-31,Sell CB,AMC,900,5.51240,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,8,2023-02-03,Sell,AMC,1000,6.19010,6190.10,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [45]:
# # fct call
return_list <- update_buys_sells(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)



 There is Buy carryforward. Update buys  100 
the s2cf_quan is:  900 
the s2cf_pps is  6.68255 
printing updated buys[1 ], dim of buys increases with c/f  


[1] 23 23

[1] 19 23

#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [46]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
s1

 Contains buy carryforward - remove buys c/f row and DO snip sells 


[1] 22 23

[1] 18 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,2023-01-20,Sell CF,AMC,100,5.5124,5.5124,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [47]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<lgl>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<lgl>
3,2023-01-20,Buy,AMC,1000,5.625,5625,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.6250,562.50,FALSE
1,2022-12-31,Buy,AMC,100,4.070,407,100,FALSE,0,⋯,4.070,407.0,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


#### <font color = lime> QC: Loop 2 review buys & sells

In [48]:
dim(buys)
dim(sells)
head(buys,4)
head(sells,4)

[1] 22 23

[1] 18 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,3,2023-01-20,Buy CF,AMC,100,5.6250,562.5,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,9,2023-02-10,Buy,AMC,2000,4.7450,9490.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,3,2023-01-20,Sell CF,AMC,100,5.5124,5.5124,0,TRUE,0,⋯,0,0,0,0,0,0,0,0,0,0
3,4,2023-02-03,Sell,AMC,900,5.5124,4961.1600,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.2550,6255.0000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,8,2023-02-03,Sell,AMC,1000,6.1901,6190.1000,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


In [49]:
# stopps

# <font color = cyan> Proof of Concept above. End of loop 2

#### <font color = yellow>1. Create data & dfs

In [50]:
b <- buys[1,]
s <- sells[1,]

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [51]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

# dim(updated_b)
# dim

proof 3 
proof 
POC - updated_s when b == s:  EQ SOLD 


### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [52]:
sale_records <- sale_records %>% add_row(updated_b , .before = 1)
# sale_records[1,]
# sale_records

#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for

In [53]:
# # fct call
return_list <- update_buys_sells(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

# dim(buys)
# dim(sells)


printing updated buys[1 ], dim of buys increases with c/f  


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [54]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
# dim(buys)
# dim(sells)
# # b1
# s1

new condtional works for filtering in copy_snip when cf-cf 


#### <font color = lime> QC: Loop 3 review buys & sells
    


In [55]:
dim(buys)
dim(sells)
head(buys, 5)
head(sells, 5)

[1] 21 23

[1] 17 23

,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,5,2023-01-23,Buy,AMC,1000,5.7880,5788.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,7,2023-01-27,Buy,AMC,2000,5.2394,10478.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,9,2023-02-10,Buy,AMC,2000,4.7450,9490.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
6,11,2023-02-10,Buy,AMC,1000,4.7800,4780.0,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
7,12,2023-02-13,Buy,AMC,3000,4.7296,14188.8,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,4,2023-02-03,Sell,AMC,900,5.5124,4961.16,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
4,6,2023-02-03,Sell,AMC,1000,6.2550,6255.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
5,8,2023-02-03,Sell,AMC,1000,6.1901,6190.10,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
6,10,2023-02-21,Sell,AMC,3000,5.8550,17565.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0
7,16,2023-02-22,Sell,AMC,6000,6.6150,39690.00,0,FALSE,0,⋯,0,0,0,0,0,0,0,0,0,0


#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [56]:
sale_records

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
3,2023-01-20,Buy CF,CFCF,100,5.625,562.5,0,TRUE,0,⋯,5.625,562.5,100,5.5124,551.24,-11.26,0,0.0000,0.00,TRUE
3,2023-01-20,Buy,AMC,1000,5.625,5625.0,900,TRUE,0,⋯,5.625,5062.5,900,5.5124,4961.16,-101.34,100,5.6250,562.50,FALSE
1,2022-12-31,Buy,AMC,100,4.070,407.0,100,FALSE,0,⋯,4.070,407.0,100,5.5124,551.24,144.24,900,5.5124,4961.16,TRUE


# <font color = cyan> Proof of Concept above. End of loop 3